# Prototype Bemchmark
For each record to be scored, the "county" specific model is read from external storage.  No model caching in memory.

* `model_load_time` (msec) is the time required to read a model file from external storage and deserialize the model object.
* `model_score_time` (msec) is the time to score one record.

## Metrics reported in this notebook are from synthetic data and **have not** been calibrated to representative dataset or model sizes.

## Notebook run-time enviornment
* **Hardware:** MacBook Pro Intel(2019), 16GB RAM, 1TB SSD drive
* **OS:** MacOS 11.6.1
* **Docker:** Docker for Desktop 4.2.0 (Mac)
* **Docker Image:** Base image: `jupyter/datascience-notebook:lab-3.2.5` with ONNX packages added

In [1]:
import glob
import os
import shutil
import pandas as pd
import numpy as np
import onnxruntime as rt
import pickle

## Setup for tests

In [2]:
# required to allow for import of project speccific utility functions
os.chdir('..')

In [3]:
# import project specific utiity functions
from utils.utils import BenchmarkDriver, load_config, actualsize_mb

In [4]:
# get configuration parameters
config = load_config('./config.yaml')
config

{'data_dir': '/Users/jim/Desktop/onnx_sandbox/data',
 'models_dir': '/Users/jim/Desktop/onnx_sandbox/models',
 'number_records': 100000,
 'number_features': 20,
 'number_informative': 14,
 'number_trees': 500,
 'fraction_for_test': 0.2,
 'number_counties': 20,
 'random_seed': 123}

In [5]:
TEST_DATA = os.path.join(config['data_dir'],'benchmark', 'test.parquet')
PERFORMANCE_DATA_DIR = os.path.join(config['data_dir'],'performance','testbed500')
MODELS_DIR = os.path.join(config['models_dir'], 'benchmark500')


In [6]:
# setup directory to collect performance data
shutil.rmtree(PERFORMANCE_DATA_DIR, ignore_errors=True)
os.makedirs(PERFORMANCE_DATA_DIR, exist_ok=True)

## Test Design

### Model Training
In an offline process, trained 20 Random Forest (RF) models using synthetic data generated by `sklearn.datasets.make_regression()` method.  These data sets have 20 explanatory variables (`X_00` to `X_19`) with a target variable called 'y'.  With the exception of setting `random_state` and `n_estimators=500` parameter to a known value, all other hyper-parameters were allowed to take their default value.

In [7]:
pd.read_parquet(os.path.join(TEST_DATA)).head()

,county,X_00,X_01,X_02,X_03,X_04,X_05,X_06,X_07,X_08,...,X_11,X_12,X_13,X_14,X_15,X_16,X_17,X_18,X_19,y
0,cnty0005,-0.047924,0.876722,-1.691040,2.626353,0.663601,0.667906,0.707864,1.312462,1.974233,...,-0.736528,-0.691200,-0.903873,-1.193744,0.687999,-0.800075,-1.300539,-0.639801,0.175235,93.457985
1,cnty0010,-0.401803,-0.685433,-0.823452,-0.191975,-0.232961,0.288845,0.599367,-1.502481,-0.917875,...,1.726174,0.693494,1.264482,1.459226,-0.508734,0.324457,1.483680,-0.582111,-0.202034,202.375458
2,cnty0000,-0.699637,0.310477,-0.535438,-0.361965,0.234813,-0.303082,-0.433491,-1.283665,0.634701,...,0.599885,-0.931460,-1.013379,0.504252,-0.556672,0.119437,1.545638,-1.011144,-0.343707,-73.529205
3,cnty0012,-0.810314,0.612156,-0.563249,0.461720,0.393322,0.873600,-0.676884,0.017982,1.143867,...,-0.559798,-0.538230,0.145180,-0.487649,1.367196,0.176917,-1.886566,1.497640,0.867901,-93.428177
4,cnty0003,-0.063003,-0.254967,0.643265,2.217894,0.429902,1.054095,0.624055,1.037485,-0.754566,...,-0.610848,-0.195606,1.082975,-0.200524,-0.438538,-1.547880,-0.146857,1.458846,-0.724704,-7.727258


For each Random Forest model, saved two model files: sklearn (.pkl file extension) and onnx (.onnx file extension).  These files are indentified by the `county` data used to generate the RF model.  Here is a sample of saved model files.

In [8]:
os.listdir(MODELS_DIR)[:10]

['cnty0000.onnx',
 'cnty0000.pkl',
 'cnty0001.onnx',
 'cnty0001.pkl',
 'cnty0002.onnx',
 'cnty0002.pkl',
 'cnty0003.onnx',
 'cnty0003.pkl',
 'cnty0004.onnx',
 'cnty0004.pkl']

### Model Scoring
For this test selected 100 random records from the test data.  For each record performed the following:
```
# Process test batch
while there are input records:
    Read one record
    Based on 'county' value, load the RF model for that 'county'  
    Record time to load and make model useable
    Record process memory RSS value
    Score the record
    Record time to score the record

# record collected metrics
Write collected run-time metrics to an external file.
```
No explicit caching of model objects were done in this test.

## Analysis of RF Tree Structure


In [9]:
# collect data on RF tree structure
tree_metrics = []
rf_models = glob.glob(os.path.join(MODELS_DIR, '*.pkl'))
for model in rf_models:
    # get file sizes
    fp_parts = os.path.splitext(model)
    metrics = {'model': fp_parts[0].split('/')[-1]}
    metrics['sklearn_file_size_mb'] = os.path.getsize(model) / (1024 * 1024)
    metrics['onnx_file_size_mb'] = os.path.getsize(fp_parts[0] + '.onnx') / (1024 * 1024)
    
    # extract tree structure
    with open(model, 'rb') as f:
        rf = pickle.load(f)
    metrics['number_of_trees'] = len(rf.estimators_)
    tree_depth = [tree.tree_.max_depth for tree in rf.estimators_]
    metrics['tree_min_depth'] = np.min(tree_depth)
    metrics['tree_max_depth'] = np.max(tree_depth)
    metrics['tree_mean_depth'] = np.mean(tree_depth)
    
    del rf
        
    # collect metrics
    tree_metrics.append(metrics)

In [10]:
# overview of tree structure
tree_metrics_df = pd.DataFrame(tree_metrics)
tree_metrics_df

,model,sklearn_file_size_mb,onnx_file_size_mb,number_of_trees,tree_min_depth,tree_max_depth,tree_mean_depth
0,cnty0000,156.392322,97.806030,500,19,27,22.504
1,cnty0001,154.341662,96.520756,500,19,27,22.068
2,cnty0002,152.485583,95.357236,500,20,28,22.172
3,cnty0003,156.238269,97.709489,500,20,30,22.718
4,cnty0004,152.140979,95.141204,500,19,27,22.258
5,cnty0005,155.043933,96.960824,500,19,29,22.376
6,cnty0006,152.872180,95.599707,500,19,29,22.446
7,cnty0007,152.600574,95.429420,500,19,27,22.066
8,cnty0008,155.510486,97.253286,500,19,29,22.376
9,cnty0009,152.872180,95.599447,500,19,28,22.598


In [11]:
tree_metrics_of_interest = ['sklearn_file_size_mb', 'onnx_file_size_mb',
                           'number_of_trees', 'tree_min_depth', 'tree_mean_depth', 'tree_max_depth']
tree_metrics_df[tree_metrics_of_interest].describe().T

,count,mean,std,min,25%,50%,75%,max
sklearn_file_size_mb,20.0,154.426062,2.117732,151.755725,152.804278,153.957141,155.539111,159.798450
onnx_file_size_mb,20.0,96.573636,1.327537,94.899602,95.556941,96.279709,97.271222,99.941508
number_of_trees,20.0,500.000000,0.000000,500.000000,500.000000,500.000000,500.000000,500.000000
tree_min_depth,20.0,19.300000,0.470162,19.000000,19.000000,19.000000,20.000000,20.000000
tree_mean_depth,20.0,22.400000,0.222541,22.066000,22.236500,22.385000,22.530500,22.828000
tree_max_depth,20.0,28.200000,1.151658,27.000000,27.000000,28.000000,29.000000,31.000000


## Model Scoring Test

### Get data for test

In [12]:
test_df = pd.read_parquet(TEST_DATA)
test_df = test_df.sample(n=100, random_state=config['random_seed'])
test_df.reset_index(inplace=True)
test_df.shape

(100, 23)

In [13]:
test_df[['index','county', 'y']].head(10)

,index,county,y
0,12136,cnty0015,-31.640778
1,16812,cnty0004,123.967995
2,10072,cnty0011,-59.197975
3,5850,cnty0010,-154.407074
4,4320,cnty0014,5.997156
5,4429,cnty0003,37.982521
6,11744,cnty0011,-19.720573
7,12106,cnty0011,271.501892
8,4230,cnty0019,140.779358
9,16858,cnty0002,45.374973


In [14]:
metrics_of_interest = ['model_process_rss_mb', 'model_load_time_ms', 'model_score_time_ms', 'model_prediction_time_ms']

### sklearn Model Scoring

In [15]:
%%time
sklearn_driver = BenchmarkDriver(
    model_type='sklearn',
    models_dir=MODELS_DIR,
    performance_fp=os.path.join(PERFORMANCE_DATA_DIR, 'sklearn_benchmark.csv'),
    test_scenario='county-level'
)

# iterate over each row and collect run-time performance statistics
for idx, row in test_df.iterrows():
    one_record = pd.DataFrame(row).T
    sklearn_driver.score_one_record(row['county'], row['index'],
            one_record.drop(['index', 'county', 'y'], axis='columns'))
    


CPU times: user 10.2 s, sys: 7.61 s, total: 17.8 s
Wall time: 16.5 s


In [16]:
# display collected performance metrics
sklearn_metrics_df = pd.read_csv(os.path.join(PERFORMANCE_DATA_DIR, 'sklearn_benchmark.csv'))
sklearn_metrics_df['model_prediction_time_ms'] = sklearn_metrics_df['model_load_time_ms'] + sklearn_metrics_df['model_score_time_ms']
sklearn_metrics_df.head(10)

,county_id,record_id,test_scenario,model_load_time_ms,model_process_rss_mb,model_score_time_ms,predicted_score,model_prediction_time_ms
0,cnty0015,12136,county-level,80.2201,536.105469,120.3641,-110.873899,200.5842
1,cnty0004,16812,county-level,60.9831,536.386719,100.4521,167.356637,161.4352
2,cnty0011,10072,county-level,82.4559,536.570312,132.8140,-26.419206,215.2699
3,cnty0010,5850,county-level,81.1715,536.609375,95.4774,-118.903737,176.6489
4,cnty0014,4320,county-level,58.8967,536.640625,108.1082,61.026405,167.0049
5,cnty0003,4429,county-level,64.0873,536.664062,92.9222,41.992953,157.0095
6,cnty0011,11744,county-level,56.3978,536.679688,112.7633,-20.342580,169.1611
7,cnty0011,12106,county-level,51.5287,536.691406,92.6941,133.894013,144.2228
8,cnty0019,4230,county-level,61.2735,536.691406,106.0864,99.426442,167.3599
9,cnty0002,16858,county-level,63.6287,536.699219,99.1003,72.667956,162.7290


In [17]:
sklearn_metrics_df[metrics_of_interest].describe(percentiles=[.25, .5, .75, .9]).T

,count,mean,std,min,25%,50%,75%,90%,max
model_process_rss_mb,100.0,530.342813,2.477784,529.199219,529.256836,529.304688,529.339844,536.133594,536.699219
model_load_time_ms,100.0,55.906134,6.388666,50.477800,51.777125,53.599950,57.502250,63.203680,82.455900
model_score_time_ms,100.0,102.461518,7.924337,85.524100,97.225175,102.073350,106.826650,112.808690,132.814000
model_prediction_time_ms,100.0,158.367652,11.671976,142.960100,151.216300,156.701850,162.752775,170.594960,215.269900


### onnx scoring test

In [18]:
%%time
onnx_driver = BenchmarkDriver(
    model_type='onnx',
    models_dir=MODELS_DIR,
    performance_fp=os.path.join(PERFORMANCE_DATA_DIR, 'onnx_benchmark.csv'),
    test_scenario='county-level'
)

# iterate over each row and collect run-time performance statistics
for idx, row in test_df.iterrows():
    one_record = pd.DataFrame(row).T
    onnx_driver.score_one_record(row['county'], row['index'],
            one_record.drop(['index', 'county', 'y'], axis='columns').astype(np.float32).to_numpy())


CPU times: user 5min 5s, sys: 12.5 s, total: 5min 17s
Wall time: 5min 20s


In [19]:
# display collected performance metrics
onnx_metrics_df = pd.read_csv(os.path.join(PERFORMANCE_DATA_DIR, 'onnx_benchmark.csv'))
onnx_metrics_df['model_prediction_time_ms'] = onnx_metrics_df['model_load_time_ms'] + onnx_metrics_df['model_score_time_ms']
onnx_metrics_df.head(10)

,county_id,record_id,test_scenario,model_load_time_ms,model_process_rss_mb,model_score_time_ms,predicted_score,model_prediction_time_ms
0,cnty0015,12136,county-level,5484.0060,1452.929688,0.5376,-110.873856,5484.5436
1,cnty0004,16812,county-level,3588.8860,1494.500000,1.0748,167.356610,3589.9608
2,cnty0011,10072,county-level,3462.4063,1506.078125,0.4653,-26.419207,3462.8716
3,cnty0010,5850,county-level,3388.9362,1503.328125,5.4532,-118.903810,3394.3894
4,cnty0014,4320,county-level,3360.2413,1505.550781,0.4730,61.026405,3360.7143
5,cnty0003,4429,county-level,3528.4494,1520.621094,0.4779,41.992980,3528.9273
6,cnty0011,11744,county-level,2767.5025,1506.078125,0.5937,-20.342573,2768.0962
7,cnty0011,12106,county-level,2763.3148,1506.078125,0.4242,133.893940,2763.7390
8,cnty0019,4230,county-level,3547.9063,1535.003906,6.5407,99.426450,3554.4470
9,cnty0002,16858,county-level,3518.2185,1527.492188,0.4964,72.667940,3518.7149


In [20]:
onnx_metrics_df[metrics_of_interest].describe(percentiles=[.25, .5, .75, .9]).T

,count,mean,std,min,25%,50%,75%,90%,max
model_process_rss_mb,100.0,1511.653477,12.287213,1452.929688,1505.260742,1512.900391,1517.093750,1521.601953,1543.324219
model_load_time_ms,100.0,3031.216202,416.083785,2691.953600,2799.115275,2856.479200,3028.126825,3601.905410,5484.006000
model_score_time_ms,100.0,0.744067,1.222314,0.378900,0.457225,0.491850,0.533375,0.598060,9.410400
model_prediction_time_ms,100.0,3031.960269,416.398868,2692.450100,2799.637650,2857.045150,3028.643250,3610.430730,5484.543600


### Differences in sklearn vs onnx predictions

In [21]:
differences = np.abs(sklearn_metrics_df['predicted_score'] - onnx_metrics_df['predicted_score'])
differences.describe()

count    1.000000e+02
mean     3.712629e-05
std      4.187545e-05
min      3.682930e-09
25%      1.015700e-05
50%      2.396663e-05
75%      4.961391e-05
max      2.575055e-04
Name: predicted_score, dtype: float64